In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory ../input/train-images-namepkl/train_feature_images_name.pkl
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory  ../input/train-feature/bk2_train_vgg_feature.pkl

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.applications import vgg16
from keras.preprocessing.image import load_img,img_to_array
from keras.models import Model
from keras.applications.imagenet_utils import preprocess_input
from PIL import Image
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance
import pandas as pd

In [ ]:
# import requests

# url = 'https://storage.googleapis.com/tensorflow/keras-applications/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels.h5'
# r = requests.get(url, allow_redirects=True)
# open('./vgg16_weights_tf_dim_ordering_tf_kernels.h5', 'wb').write(r.content)

In [ ]:
imgs_path ="../input/shopee-product-matching/train_images/"

In [ ]:
train_csv_df=pd.read_csv('../input/shopee-product-matching/train.csv')
test_csv_df=pd.read_csv('../input/shopee-product-matching/test.csv')

# print(train_csv_df.head())
# print(test_csv_df.head())

In [ ]:
# imgs_path = "style/"
imgs_model_width, imgs_model_height = 224, 224
nb_closest_images = 5 # number of most similar images to retrieve
# load the model
# vgg_model = vgg16.VGG16(weights='imagenet', include_top=False)
# vgg_model = vgg16.VGG16(weights = None)
vgg_model = vgg16.VGG16(weights = '../input/vgg-weight-1/vgg16_weights_tf_dim_ordering_tf_kernels.h5')
# vgg_model.load_weights('./vgg16_weights_tf_dim_ordering_tf_kernels.h5')

# remove the last layers in order to get features instead of predictions
feat_extractor = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer("fc2").output)

# print the layers of the CNN
feat_extractor.summary()

In [ ]:
files = [x for x in os.listdir(imgs_path)]
# print("number of images:",len(files))

In [ ]:
# # load an image in PIL format
# original = load_img(imgs_path+files[0], target_size=(imgs_model_width, imgs_model_height))
# plt.imshow(original)
# plt.show()
# print("image loaded successfully!")

In [ ]:
# save to file
from pickle import dump,load
import pickle
import copy

In [ ]:
importedImages = []
importedImages_names = []
counter = 0
imgs_features_pkl=[]
imgs_names_pkl=[]

test_imgs_features_pkl=[]
test_imgs_features_name_pkl=[]

In [ ]:
###################3 TEST
posting_ids=[]
for item in test_csv_df[['posting_id','image']].values.tolist():
    # print(item[0],item[1])
    given_img='../input/shopee-product-matching/test_images/'+item[1]
#     print(f)
    counter=counter+1
    # filename = f
    original = load_img(given_img, target_size=(224, 224))
    numpy_image = img_to_array(original)
    image_batch = np.expand_dims(numpy_image, axis=0)
    importedImages.append(image_batch)
    test_imgs_features_name_pkl.append(item[1])
    if counter==1000:
        images = np.vstack(importedImages)
        processed_imgs = preprocess_input(images.copy())
        # extract the images features
        imgs_features = feat_extractor.predict(processed_imgs)
        if len(test_imgs_features_pkl)>0:
            test_imgs_features_pkl = np.append(test_imgs_features_pkl, imgs_features, axis=0)
#             print(len(test_imgs_features_pkl))
            counter=0
            importedImages = []
        else:
            images = np.vstack(importedImages)
            processed_imgs = preprocess_input(images.copy())
          # extract the images features
            test_imgs_features_pkl = feat_extractor.predict(processed_imgs)
            counter=0
            importedImages = []

In [ ]:
remaining_count=len(test_imgs_features_pkl)
# print(remaining_count)
# print(len(test_imgs_features_name_pkl))
test_imgs_features_name_pkl=test_imgs_features_name_pkl[:remaining_count]
# print(len(test_imgs_features_name_pkl))
# print(len(test_csv_df[['posting_id','image']].values.tolist()))
rmaining_images = test_csv_df[['posting_id','image']].values.tolist()[remaining_count:]
# print(rmaining_images)
# print(len(rmaining_images))
counter=0

In [ ]:
###################3 TEST
posting_ids=[]
importedImages = []
importedImages_names = []
for item in rmaining_images:
    # print(item[0],item[1])
    given_img='../input/shopee-product-matching/test_images/'+item[1]
    # print(f)
    counter=counter+1
    # filename = f
    original = load_img(given_img, target_size=(224, 224))
    numpy_image = img_to_array(original)
    image_batch = np.expand_dims(numpy_image, axis=0)
    importedImages.append(image_batch)
    test_imgs_features_name_pkl.append(item[1])
    if counter==len(rmaining_images):
        images = np.vstack(importedImages)
        processed_imgs = preprocess_input(images.copy())
        # extract the images features
        imgs_features = feat_extractor.predict(processed_imgs)
        if len(test_imgs_features_pkl)>0:
#             print('if')
            test_imgs_features_pkl = np.append(test_imgs_features_pkl, imgs_features, axis=0)
#             print(len(test_imgs_features_pkl))
            counter=0
            importedImages = []
        else:
#             print('else')
            test_imgs_features_pkl = imgs_features
#             print(len(test_imgs_features_pkl))
            counter=0
            importedImages = []

In [ ]:
imgs_features_pkl=[]
importedImages_names = []
with open("../input/train-feature/bk2_train_vgg_feature.pkl", 'rb') as file:
    imgs_features_pkl = load(file)
    print(len(imgs_features_pkl))

with open("../input/train-images-namepkl/train_feature_images_name.pkl", 'rb') as file:
    importedImages_names = load(file)
    print(len(importedImages_names))

In [ ]:
importedImages_names111 = np.append(importedImages_names,test_imgs_features_name_pkl,  axis=0)
imgs_features_pkl111 = np.append(imgs_features_pkl,test_imgs_features_pkl,  axis=0)

In [ ]:
cosSimilarities = cosine_similarity(imgs_features_pkl111)
posting_ids=[]
# print(len(cosSimilarities))
cos_similarities_df = pd.DataFrame(cosSimilarities, columns=importedImages_names111, index=importedImages_names111)
# print(cos_similarities_df.head(5))

In [ ]:
for item in test_csv_df[['posting_id','image']].values.tolist():
#     print(item[0],item[1])
    given_img=item[1]
    closest_imgs = cos_similarities_df[given_img].sort_values(ascending=False)[1:nb_closest_images+1].index
    closest_imgs_scores = cos_similarities_df[given_img].sort_values(ascending=False)[1:nb_closest_images+1]
    temp_list=[]
    temp_list.append(item[0])
    for i in range(0,len(closest_imgs)):
        original = load_img('../input/shopee-product-matching/train_images/'+closest_imgs[i], target_size=(imgs_model_width, imgs_model_height))
        posting=train_csv_df[train_csv_df['image']==closest_imgs[i]]
#         print(list(posting['posting_id']))
        temp_list.append(list(posting['posting_id'])[0])
      # plt.savefig('')
#         plt.imshow(original)
#         plt.show()
#         print("similarity score : ",closest_imgs_scores[i])
    posting_ids.append([item[0]," ".join(temp_list)])


In [ ]:
posting_id_df= pd.DataFrame(posting_ids,columns=['posting_id','matches'])
posting_id_df.to_csv("./submission.csv" ,index=False)

In [ ]:
# posting_ids=[]
# for item in test_csv_df[['posting_id','image']].values.tolist():
# #     print(item[0],item[1])
#     given_img='../input/shopee-product-matching/test_images/'+item[1]
#   #########################################3
#     cosSimilarities=None
#     imgs_features=None
#     original = load_img(given_img, target_size=(imgs_model_width, imgs_model_height))
# #     plt.imshow(original)
# #     plt.show()
# #     print("image loaded successfully!")
#     # convert the PIL image to a numpy array
#     # in PIL - image is in (width, height, channel)
#     # in Numpy - image is in (height, width, channel)
#     numpy_image1 = img_to_array(original)
#     # convert the image / images into batch format
#     # expand_dims will add an extra dimension to the data at a particular axis
#     # we want the input matrix to the network to be of the form (batchsize, height, width, channels)
#     # thus we add the extra dimension to the axis 0.
#     image_batch1 = np.expand_dims(numpy_image1, axis=0)
#     # prepare the image for the VGG model
#     processed_image1 = preprocess_input(image_batch1.copy())
#     img_features111 = feat_extractor.predict(processed_image1)
#     counter=0
#     cosSimilarities_1={}
#     for img_train in imgs_features_pkl:
#         result = cosine_similarity(img_features111, img_train.reshape(1,4096))
#         if result > 0.65:
#             cosSimilarities_1[str(result[0][0])] = importedImages_names[counter]
#         counter +=1
#     update_list = sorted(cosSimilarities_1.items(),reverse=True)
# #     print(update_list)
#   ##############################################
#     temp_list=[]
#     temp_list.append(item[0])
#     for k in update_list[:10]:
# #         print(k)
#         original = load_img('../input/shopee-product-matching/train_images/'+k[1], target_size=(imgs_model_width, imgs_model_height))
#         posting=train_csv_df[train_csv_df['image']==k[1]]
# #         print(list(posting['posting_id']))
#         temp_list.append(list(posting['posting_id'])[0])
#         # plt.savefig('')
# #         plt.imshow(original)
# #         plt.show()
#         # print("image loaded successfully!")
# #         print("similarity score : ",k[0])
#     posting_ids.append([item[0]," ".join(temp_list)])

In [ ]:
# posting_id_df= pd.DataFrame(posting_ids,columns=['posting_id','matches'])
# # " ".join(list(posting_id_df['matches'][0]))
# posting_id_df.to_csv("./submission.csv" ,index=False)